In [1]:
# load Preliminaries
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import networkx as nx
import os
from glob import glob

In [ ]:
# !!!!!!! think to run 'python main.py -e draw' with the save_json activated !!!!!

In [2]:
###### Parameters #######
DATASETS = [{'name': 'letter', 'threshold': 1},
            {'name': 'AIDS', 'threshold': 1},
            {'name': 'mutagenicity', 'threshold': 5}]
idx = 1
dataset = DATASETS[idx]['name']
threshold = DATASETS[idx]['threshold']
folder = os.path.join('../results/draw/', dataset, '')
experiment = 'pagerank'

In [3]:
###### Data ######
graphs = []
print
for filename in glob(f'{folder}*.json'):
    if not filename.split('/')[-1].startswith(experiment):
        continue
    print(filename)
    with open(filename, mode='r') as fp:
        tmp_graph = json.load(fp)
        graphs.append(tmp_graph)


# print(*graphs, sep='\n\n')

../results/draw/AIDS/pagerank_molid669986.json
../results/draw/AIDS/pagerank_molid646645.json
../results/draw/AIDS/pagerank_molid369774.json
../results/draw/AIDS/pagerank_molid663915.json
../results/draw/AIDS/pagerank_molid686667.json
../results/draw/AIDS/pagerank_molid634193.json
../results/draw/AIDS/pagerank_molid30152.json
../results/draw/AIDS/pagerank_molid680831.json
../results/draw/AIDS/pagerank_molid684035.json
../results/draw/AIDS/pagerank_molid646232.json


In [4]:
# loading the engines locally along with plug-in.
HTML('''
<script src="./sigma.min.js"></script>
''')

In [5]:
if dataset == 'mutagenicity':
    for graph in graphs:
        nx_graph = nx.Graph()
        for node in graph['nodes']:
            nx_graph.add_node(node['id'])
    
        for edge in graph['edges']:
            nx_graph.add_edge(edge['source'], edge['target'], weight=int(edge['label']))
    
        # pos = nx.drawing.layout.spring_layout(nx_graph, iterations=2500, center=[0.0, 0.0], seed=42)
        pos = nx.drawing.layout.kamada_kawai_layout(nx_graph)

        for node in graph['nodes']:
            node['x'], node['y'] = pos[node['id']]

In [6]:
js_text_template = Template('''

    var g = $graph_data ;

s = new sigma({graph: g, container: '$container', settings: { defaultNodeColor: '#ec5148', labelThreshold: $threshold, zoomingRatio:1.4} });

s.graph.nodes().forEach(function(n) {
  n.originalColor = n.color;
});
s.graph.edges().forEach(function(e) {
  e.originalColor = e.color;
});

s.bind('clickNode', function(e) {
  var nodeId = e.data.node.id,
      toKeep = s.graph.neighbors(nodeId);
  toKeep[nodeId] = e.data.node;

  s.graph.nodes().forEach(function(n) {
    if (toKeep[n.id])
      n.color = n.originalColor;
    else
      n.color = '#eee';
  });

  s.graph.edges().forEach(function(e) {
    if (toKeep[e.source] && toKeep[e.target])
      e.color = e.originalColor;
    else
      e.color = '#eee';
  });

  s.refresh();
});

s.bind('clickStage', function(e) {
  s.graph.nodes().forEach(function(n) {
    n.color = n.originalColor;
  });

  s.graph.edges().forEach(function(e) {
    e.color = e.originalColor;
  });

  s.refresh();
});

s.refresh();

// Listeners
var force = false;
document.getElementById('layout').onclick = function() {
  if (!force)
    s.startForceAtlas2({worker: true});
  else
    s.stopForceAtlas2();
  force = !force;
};

document.getElementById('export').onclick = function() {
  console.log('exporting...');
  var output = s.toSVG({download: true, filename: '$filename', size: 1000, labels: true});
  // console.log(output);
};

// Initialize the dragNodes plugin:
var dragListener = sigma.plugins.dragNodes(s, s.renderers[0]);

dragListener.bind('startdrag', function(event) {
  console.log(event);
});
dragListener.bind('drag', function(event) {
  console.log(event);
});
dragListener.bind('drop', function(event) {
  console.log(event);
});
dragListener.bind('dragend', function(event) {
  console.log(event);
});


''')

In [7]:
html_template = Template('''

<div id="graph-div" style="height:800px"></div>
<script> $js_text </script>
''')

In [8]:
idx_graph = 5
graph_data = graphs[idx_graph]
js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div',
                                       'threshold': threshold,
                                       'filename': 'test.svg'
                                      })
HTML(html_template.substitute({'js_text': js_text}))